# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding = "utf-8" )
weather_data = weather_data.rename(columns = {"Longtidue":"Longitude"})
weather_data.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Hirānagar,92,IN,2020-03-14 23:38:56,57,32.45,75.27,52.20,3.60
1,Karratha,90,AU,2020-03-14 23:38:56,72,-20.74,116.85,83.50,16.37
2,Asifābād,59,IN,2020-03-14 23:38:56,61,19.37,79.28,77.14,1.45
3,Ponta do Sol,75,PT,2020-03-14 23:38:56,67,32.67,-17.10,60.80,10.29
4,Upernavik,98,GL,2020-03-14 23:38:56,98,72.79,-56.15,-14.10,3.60


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [4]:

fig = gmaps.figure()
#Make heatmap based off off the max value of humidity for the given cities
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=(humidity.max()),
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions:
    * Max Temp greater than 70 but lower than 80
    * Wind speed less than 10 mph
    * Cloudiness less than 5%
* Drop any rows will null values. 

In [10]:
#Filter the weather data by the desired criteria
ideal_weather = weather_data.loc[(weather_data['Max Temp'] < 80)]
ideal_weather = ideal_weather.loc[(weather_data['Max Temp'] > 70)]
ideal_weather = ideal_weather.loc[(weather_data['Wind Speed'] < 10)]
ideal_weather = ideal_weather.loc[weather_data['Cloudiness'] < 5 ]

#Drop any null values and reset the index
hotel_df = ideal_weather.dropna().reset_index(drop=True)
#Prints the number of hotels that satisfy the criteria
print(len(hotel_df))

8


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
#Add Hotel Name to DataFrame
hotel_df["Hotel Name"] = ""
hotel_df.head()

,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Avanigadda,0,IN,2020-03-14 23:38:56,85,16.02,80.92,73.44,5.06,
1,San Felipe,4,VE,2020-03-14 23:38:56,97,10.34,-68.74,70.63,2.26,
2,Goundi,0,TD,2020-03-14 23:38:56,11,9.36,17.37,74.86,5.14,
3,Kaeo,0,NZ,2020-03-14 23:38:56,61,-35.10,173.78,75.99,1.99,
4,Hingoli,0,IN,2020-03-14 23:38:56,18,19.72,77.15,70.57,8.16,


In [14]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set parameters to search for hotel
params = {
        "radius": 5000,
        "key": g_key,
        "keyword" : "hotel",
        "type": "lodging"
    }
for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(url,params).json()
    print(f"Searching for hotel near {row['Name']}...")
    try:
        print(f"Found the hotel {response['results'][0]['name']}. Adding to list")
        hotel_df.loc[index, "Hotel Name"] =(response['results'][0]['name'])
    except:
        print("Hotel not found")
        hotel_df.loc[index,"Hotel Name"] = "No Nearby Hotel Found"
hotel_df

Searching for hotel near Avanigadda...
Hotel not found
Searching for hotel near San Felipe...
Found the hotel Hotel La Antigua Mision. Adding to list
Searching for hotel near Goundi...
Hotel not found
Searching for hotel near Kaeo...
Found the hotel Pacific Harbour Lodge. Adding to list
Searching for hotel near Hingoli...
Found the hotel Hotel Dev Palace Lodging. Adding to list
Searching for hotel near Kontagora...
Found the hotel Had Resources Hotel. Adding to list
Searching for hotel near Avera...
Hotel not found
Searching for hotel near Marawi...
Found the hotel Marawi Resort Hotel. Adding to list


,Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
0,Avanigadda,0,IN,2020-03-14 23:38:56,85,16.02,80.92,73.44,5.06,No Nearby Hotel Found
1,San Felipe,4,VE,2020-03-14 23:38:56,97,10.34,-68.74,70.63,2.26,Hotel La Antigua Mision
2,Goundi,0,TD,2020-03-14 23:38:56,11,9.36,17.37,74.86,5.14,No Nearby Hotel Found
3,Kaeo,0,NZ,2020-03-14 23:38:56,61,-35.10,173.78,75.99,1.99,Pacific Harbour Lodge
4,Hingoli,0,IN,2020-03-14 23:38:56,18,19.72,77.15,70.57,8.16,Hotel Dev Palace Lodging
5,Kontagora,1,NG,2020-03-14 23:38:56,11,10.40,5.47,76.98,7.83,Had Resources Hotel
6,Avera,1,US,2020-03-14 23:38:56,48,33.19,-82.53,78.80,6.93,No Nearby Hotel Found
7,Marawi,0,PH,2020-03-14 23:38:56,81,8.00,124.29,77.00,3.00,Marawi Resort Hotel


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations_h = hotel_df[["Latitude", "Longitude"]]

In [16]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations_h, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=3,
    info_box_content=[x for x in hotel_info])

fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
    
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))